In [1]:
# 各種ライブラリをインポート

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import matplotlib.pyplot as plt

import datetime

In [3]:
# GPUの利用可否

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda


In [5]:
# data-loader

In [6]:
image_size = 112
degress1 = 30
degress2 = 60

data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degress1),
    transforms.RandomRotation(degress2),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomErasing(value='random')
])

In [7]:
# dir_loader

In [8]:
batch_size = 100
num_workers = 10

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

#trainset = torchvision.datasets.ImageFolder(root='./dataset/tiny-imagenet-200/train', transform=data_transform, target_transform=None, is_valid_file=None)
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
#testset = torchvision.datasets.ImageFolder(root='./dataset/tiny-imagenet-200/test', transform=data_transform, target_transform=None, is_valid_file=None)
#testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
dataloaders_dict = {
    "train" : trainloader,
    "val" : testloader
}

In [10]:
flatten_features = 7 * 7 * 512

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        # Block1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.norm1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop1 = nn.Dropout(p=0.5)
        # Block2
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.norm2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop2 = nn.Dropout(p=0.5)
        # Block3
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.norm3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop3 = nn.Dropout(p=0.5)
        # Block4
        self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.norm4 = nn.BatchNorm2d(512)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop4 = nn.Dropout(p=0.5)
        # Block5
        self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.norm5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.drop5 = nn.Dropout(p=0.5)
        # fc
        self.fc1 = nn.Linear(in_features=flatten_features, out_features=4096)
        self.fc2 = nn.Linear(in_features=4096, out_features=4096)
        self.drop6 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(in_features=4096, out_features=10)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, x):
        # Block1
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.norm1(x)
        x = self.pool1(x)
        #x = self.drop1(x)
        # Block2
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.norm2(x)
        x = self.pool2(x)
        #x = self.drop2(x)
        # Block3
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.norm3(x)
        x = self.pool3(x)
        #x = self.drop3(x)
        # Block4
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.norm4(x)
        x = self.pool4(x)
        #x = self.drop4(x)
        # Block5
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.norm5(x)
        x = self.pool5(x)
        x = self.drop5(x)
        # fc
        x = self.avgpool(x)
        #x = x.view(-1, flatten_features)
        #x = x.view(x.size(0),-1)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.drop6(x)
        x = F.relu(self.fc2(x))
        x = F.softmax(input=self.fc3(x), dim=0)
        return x


net = VGG16()
net = net.to(device)
net = torch.nn.DataParallel(net)
#net.cuda()

In [11]:
# Optimazerの設定

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
# 学習

In [ ]:
print(len(trainloader))

epoch_count = 100
#check_point = 100

for epoch in range(epoch_count):  # loop over the dataset multiple times
    print('----------')
    print('Epoch {}/{} {}'.format(epoch+1, epoch_count, datetime.datetime.now()))

    for phase in ['train', 'val']:
        if phase == 'train':
            net.train()
        else:
            net.eval()
      
        epoch_loss = 0.0
        epoch_corrects = 0

        if (epoch == 0) and (phase == 'train'):
            continue
      
#      for inputs, labels in tqdm(dataloaders_dict[phase]):
        for i, data in enumerate(dataloaders_dict[phase], 0):
  
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)

        epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        
print('Finished Training')

500
----------
Epoch 1/100 2020-01-03 10:57:11.947532
val Loss: 2.3026 Acc: 0.1080
----------
Epoch 2/100 2020-01-03 10:57:21.749148
train Loss: 2.3023 Acc: 0.1114
val Loss: 2.3012 Acc: 0.1667
----------
Epoch 3/100 2020-01-03 10:59:08.400627
train Loss: 2.2929 Acc: 0.1630
val Loss: 2.2886 Acc: 0.1899
----------
Epoch 4/100 2020-01-03 11:00:55.594965
train Loss: 2.2852 Acc: 0.1933
val Loss: 2.2780 Acc: 0.2311
----------
Epoch 5/100 2020-01-03 11:02:42.619364
train Loss: 2.2713 Acc: 0.2241
val Loss: 2.2636 Acc: 0.2420
----------
Epoch 6/100 2020-01-03 11:04:29.521673
train Loss: 2.2607 Acc: 0.2472
val Loss: 2.2559 Acc: 0.2659
----------
Epoch 7/100 2020-01-03 11:06:16.805535
train Loss: 2.2533 Acc: 0.2659
val Loss: 2.2455 Acc: 0.2893
----------
Epoch 8/100 2020-01-03 11:08:03.655002
train Loss: 2.2456 Acc: 0.2827
val Loss: 2.2422 Acc: 0.2834
----------
Epoch 9/100 2020-01-03 11:09:50.556630
train Loss: 2.2398 Acc: 0.2963
val Loss: 2.2354 Acc: 0.3090
----------
Epoch 10/100 2020-01-03 11

train Loss: 2.1796 Acc: 0.4489
val Loss: 2.1796 Acc: 0.4245
----------
Epoch 76/100 2020-01-03 13:10:15.758039
train Loss: 2.1790 Acc: 0.4470
val Loss: 2.1801 Acc: 0.4184
----------
Epoch 77/100 2020-01-03 13:12:03.223027
train Loss: 2.1783 Acc: 0.4438
val Loss: 2.1788 Acc: 0.4241
----------
Epoch 78/100 2020-01-03 13:13:50.689874
train Loss: 2.1783 Acc: 0.4478
val Loss: 2.1807 Acc: 0.4187
----------
Epoch 79/100 2020-01-03 13:15:37.986046
train Loss: 2.1756 Acc: 0.4519
val Loss: 2.1776 Acc: 0.4189
----------
Epoch 80/100 2020-01-03 13:17:25.405726
train Loss: 2.1777 Acc: 0.4476
val Loss: 2.1757 Acc: 0.4310
----------
Epoch 81/100 2020-01-03 13:19:12.874872
train Loss: 2.1774 Acc: 0.4532
val Loss: 2.1792 Acc: 0.4243
----------
Epoch 82/100 2020-01-03 13:21:00.296491
train Loss: 2.1763 Acc: 0.4548
val Loss: 2.1781 Acc: 0.4286
----------
Epoch 83/100 2020-01-03 13:22:47.636138
train Loss: 2.1759 Acc: 0.4482
val Loss: 2.1779 Acc: 0.4278
----------
Epoch 84/100 2020-01-03 13:24:35.032770
t

In [ ]:
"""
running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #inputs = torch.reshape(inputs, (-1, 3, image_size, image_size))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #scheduler.setp()

        # print statistics
        running_loss += loss.item()
        if i % check_point == 0:    # print every 2000 mini-batches
            print(datetime.datetime.now())
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')
"""

In [ ]:
# テストデータによる検証

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
#plt.imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# 学習結果を保存

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)